# Multi-Agent System with Couchbase Vector Store and Amazon Bedrock

This notebook demonstrates how to create a multi-agent system that uses Couchbase as a vector store for document operations. We'll create three specialized agents:

1. Embedder Agent: Handles document storage and vector embeddings
2. Researcher Agent: Searches and retrieves information from documents
3. Content Writer Agent: Formats and presents research findings in a user-friendly way

All agents will use the Return of Control (ROC) pattern to interact with the system.

## Architecture

![Sequence Diagram](./assets/seq.png)
![Graph](./assets/graph.png)

In [1]:
import os
import json
import logging
import time
import uuid
from datetime import timedelta
from dotenv import load_dotenv

import boto3
from couchbase.auth import PasswordAuthenticator
from couchbase.cluster import Cluster
from couchbase.exceptions import (
    CouchbaseException,
    InternalServerFailureException,
    QueryIndexAlreadyExistsException,
    ServiceUnavailableException
)
from couchbase.management.buckets import CreateBucketSettings
from couchbase.management.search import SearchIndex
from couchbase.options import ClusterOptions
from langchain_aws import BedrockEmbeddings
from langchain_couchbase.vectorstores import CouchbaseVectorStore

# Setup logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Load environment variables
load_dotenv()

True

## Configuration

Set up AWS and Couchbase configuration variables.

In [2]:
# Couchbase Configuration
CB_HOST = os.getenv("CB_HOST", "couchbase://localhost")
CB_USERNAME = os.getenv("CB_USERNAME", "Administrator")
CB_PASSWORD = os.getenv("CB_PASSWORD", "password")
CB_BUCKET_NAME = os.getenv("CB_BUCKET_NAME", "vector-search-testing")
SCOPE_NAME = os.getenv("SCOPE_NAME", "shared")
COLLECTION_NAME = os.getenv("COLLECTION_NAME", "bedrock") 
INDEX_NAME = os.getenv("INDEX_NAME", "vector_search_bedrock")

# AWS Configuration
AWS_REGION = os.getenv("AWS_REGION", "us-east-1")
AWS_ACCESS_KEY_ID = os.getenv("AWS_ACCESS_KEY_ID")
AWS_SECRET_ACCESS_KEY = os.getenv("AWS_SECRET_ACCESS_KEY")
AWS_ACCOUNT_ID = os.getenv("AWS_ACCOUNT_ID")

# Initialize AWS clients
bedrock_agent_client = boto3.client(
    'bedrock-agent',
    region_name=AWS_REGION,
    aws_access_key_id=AWS_ACCESS_KEY_ID,
    aws_secret_access_key=AWS_SECRET_ACCESS_KEY
)

bedrock_client = boto3.client(
    'bedrock',
    region_name=AWS_REGION,
    aws_access_key_id=AWS_ACCESS_KEY_ID,
    aws_secret_access_key=AWS_SECRET_ACCESS_KEY
)

iam_client = boto3.client(
    'iam',
    region_name=AWS_REGION,
    aws_access_key_id=AWS_ACCESS_KEY_ID,
    aws_secret_access_key=AWS_SECRET_ACCESS_KEY
)

## Setup Couchbase Collections

Create and configure the Couchbase collections for vector storage.

In [3]:
def setup_collection(cluster, bucket_name, scope_name, collection_name):
    """Set up Couchbase collection for vector storage"""
    try:
        # Check if bucket exists, create if it doesn't
        try:
            bucket = cluster.bucket(bucket_name)
            logging.info(f"Bucket '{bucket_name}' exists.")
        except Exception:
            logging.info(f"Bucket '{bucket_name}' does not exist. Creating it...")
            bucket_settings = CreateBucketSettings(
                name=bucket_name,
                bucket_type='couchbase',
                ram_quota_mb=1024,
                flush_enabled=True,
                num_replicas=0
            )
            cluster.buckets().create_bucket(bucket_settings)
            bucket = cluster.bucket(bucket_name)
            logging.info(f"Bucket '{bucket_name}' created successfully.")

        bucket_manager = bucket.collections()

        # Check if scope exists, create if it doesn't
        scopes = bucket_manager.get_all_scopes()
        scope_exists = any(scope.name == scope_name for scope in scopes)
        
        if not scope_exists and scope_name != "_default":
            logging.info(f"Scope '{scope_name}' does not exist. Creating it...")
            bucket_manager.create_scope(scope_name)
            logging.info(f"Scope '{scope_name}' created successfully.")

        # Check if collection exists, create if it doesn't
        collections = bucket_manager.get_all_scopes()
        collection_exists = any(
            scope.name == scope_name and collection_name in [col.name for col in scope.collections]
            for scope in collections
        )

        if not collection_exists:
            logging.info(f"Collection '{collection_name}' does not exist. Creating it...")
            bucket_manager.create_collection(scope_name, collection_name)
            logging.info(f"Collection '{collection_name}' created successfully.")
        else:
            logging.info(f"Collection '{collection_name}' already exists.")

        # Wait for collection to be ready
        collection = bucket.scope(scope_name).collection(collection_name)
        time.sleep(2)

        # Ensure primary index exists
        try:
            cluster.query(f"CREATE PRIMARY INDEX IF NOT EXISTS ON `{bucket_name}`.`{scope_name}`.`{collection_name}`").execute()
            logging.info("Primary index created successfully.")
        except Exception as e:
            logging.warning(f"Error creating primary index: {str(e)}")

        return collection
    except Exception as e:
        raise RuntimeError(f"Error setting up collection: {str(e)}")

## Initialize Couchbase Connection

Connect to Couchbase and set up collections.

In [4]:
try:
    # Connect to Couchbase
    auth = PasswordAuthenticator(CB_USERNAME, CB_PASSWORD)
    options = ClusterOptions(auth)
    cluster = Cluster(CB_HOST, options)
    cluster.wait_until_ready(timedelta(seconds=5))
    logging.info("Successfully connected to Couchbase")
    
    # Set up collections
    collection = setup_collection(cluster, CB_BUCKET_NAME, SCOPE_NAME, COLLECTION_NAME)
    logging.info("Collections setup complete")
    
except Exception as e:
    raise ConnectionError(f"Failed to connect to Couchbase: {str(e)}")

2025-02-19 06:26:42,091 - INFO - Successfully connected to Couchbase
2025-02-19 06:26:43,367 - INFO - Bucket 'vector-search-testing' exists.
2025-02-19 06:26:45,662 - INFO - Collection 'bedrock' already exists.
2025-02-19 06:26:48,761 - INFO - Primary index created successfully.
2025-02-19 06:26:48,762 - INFO - Collections setup complete


## Initialize Vector Store

Set up the vector store with Bedrock embeddings.

In [5]:
try:
    # Initialize Bedrock embeddings
    embeddings = BedrockEmbeddings(
        client=bedrock_client,
        model_id="amazon.titan-embed-text-v2:0"
    )
    logging.info("Successfully created Bedrock embeddings client")
    
    # Initialize vector store
    vector_store = CouchbaseVectorStore(
        cluster=cluster,
        bucket_name=CB_BUCKET_NAME,
        scope_name=SCOPE_NAME,
        collection_name=COLLECTION_NAME,
        embedding=embeddings,
        index_name=INDEX_NAME
    )
    logging.info("Successfully created vector store")
    
except Exception as e:
    raise ValueError(f"Failed to initialize vector store: {str(e)}")

2025-02-19 06:26:48,769 - INFO - Successfully created Bedrock embeddings client
2025-02-19 06:26:52,500 - INFO - Successfully created vector store


## Create Agent Helper Functions

Functions to help create and manage our agents.

In [6]:
def create_agent_role(agent_name, model_id):
    """Create IAM role and policies for the agent"""
    policy_name = f"{agent_name}-policy"
    role_name = f"AmazonBedrockExecutionRoleForAgents_{agent_name}"
    
    # Create policy
    policy_doc = {
        "Version": "2012-10-17",
        "Statement": [
            {
                "Sid": "BedrockModelAccess",
                "Effect": "Allow",
                "Action": "bedrock:InvokeModel",
                "Resource": f"arn:aws:bedrock:{AWS_REGION}::foundation-model/{model_id}"
            }
        ]
    }
    
    try:
        policy = iam_client.create_policy(
            PolicyName=policy_name,
            PolicyDocument=json.dumps(policy_doc)
        )
    except iam_client.exceptions.EntityAlreadyExistsException:
        policy = iam_client.get_policy(PolicyArn=f"arn:aws:iam::{AWS_ACCOUNT_ID}:policy/{policy_name}")
    
    # Create role
    trust_policy = {
        "Version": "2012-10-17",
        "Statement": [{
            "Effect": "Allow",
            "Principal": {"Service": "bedrock.amazonaws.com"},
            "Action": "sts:AssumeRole"
        }]
    }
    
    try:
        role = iam_client.create_role(
            RoleName=role_name,
            AssumeRolePolicyDocument=json.dumps(trust_policy)
        )
        # Attach policy to role
        iam_client.attach_role_policy(
            RoleName=role_name,
            PolicyArn=policy['Policy']['Arn']
        )
    except iam_client.exceptions.EntityAlreadyExistsException:
        role = iam_client.get_role(RoleName=role_name)
    
    return role

def wait_for_agent_status(agent_id, target_statuses=['Available', 'PREPARED', 'NOT_PREPARED'], max_attempts=30, delay=2):
    """Wait for agent to reach any of the target statuses"""
    for attempt in range(max_attempts):
        try:
            response = bedrock_agent_client.get_agent(agentId=agent_id)
            current_status = response['agent']['agentStatus']
            
            if current_status in target_statuses:
                logging.info(f"Agent {agent_id} reached status: {current_status}")
                return current_status
            elif current_status == 'FAILED':
                logging.error(f"Agent {agent_id} failed")
                return 'FAILED'
            
            logging.info(f"Agent status: {current_status}, waiting... (attempt {attempt + 1}/{max_attempts})")
            time.sleep(delay)
            
        except Exception as e:
            logging.warning(f"Error checking agent status: {str(e)}")
            time.sleep(delay)
    
    return current_status  # Return the last known status instead of None

def create_agent(name, instructions, functions, model_id="anthropic.claude-3-sonnet-20240229-v1:0"):
    """Create a Bedrock agent with ROC action groups"""
    try:
        # Create agent role
        role = create_agent_role(name, model_id)
        
        # List existing agents
        existing_agents = bedrock_agent_client.list_agents()
        existing_agent = next(
            (agent for agent in existing_agents['agentSummaries'] 
             if agent['agentName'] == name),
            None
        )
        
        # Handle existing agent
        if existing_agent:
            agent_id = existing_agent['agentId']
            logging.info(f"Found existing agent '{name}' with ID: {agent_id}")
            
            # Check agent status
            response = bedrock_agent_client.get_agent(agentId=agent_id)
            status = response['agent']['agentStatus']
            
            if status in ['NOT_PREPARED', 'FAILED']:
                logging.info(f"Deleting agent '{name}' with status {status}")
                bedrock_agent_client.delete_agent(agentId=agent_id)
                time.sleep(10)  # Wait after deletion
                existing_agent = None
        
        # Create new agent if needed
        if not existing_agent:
            logging.info(f"Creating new agent '{name}'")
            agent = bedrock_agent_client.create_agent(
                agentName=name,
                description=f"{name.title()} agent for document operations",
                instruction=instructions,
                agentResourceRoleArn=role['Role']['Arn'],
                idleSessionTTLInSeconds=1800,
                foundationModel=model_id
            )
            agent_id = agent['agent']['agentId']
            logging.info(f"Created new agent '{name}' with ID: {agent_id}")
        else:
            agent_id = existing_agent['agentId']
        
        # Wait for initial creation if needed
        status = wait_for_agent_status(agent_id, target_statuses=['NOT_PREPARED', 'PREPARED', 'Available'])
        if status not in ['NOT_PREPARED', 'PREPARED', 'Available']:
            raise Exception(f"Agent failed to reach valid state: {status}")
        
        # Create action group if needed
        try:
            bedrock_agent_client.create_agent_action_group(
                agentId=agent_id,
                agentVersion="DRAFT",
                actionGroupExecutor={"customControl": "RETURN_CONTROL"},
                actionGroupName=f"{name}_actions",
                functionSchema={"functions": functions},
                description=f"Action group for {name} operations"
            )
            logging.info(f"Created action group for agent '{name}'")
            time.sleep(5)
        except bedrock_agent_client.exceptions.ConflictException:
            logging.info(f"Action group already exists for agent '{name}'")
        
        # Prepare agent if needed
        if status == 'NOT_PREPARED':
            try:
                logging.info(f"Starting preparation for agent '{name}'")
                bedrock_agent_client.prepare_agent(agentId=agent_id)
                status = wait_for_agent_status(
                    agent_id, 
                    target_statuses=['PREPARED', 'Available']
                )
                logging.info(f"Agent '{name}' preparation completed with status: {status}")
            except Exception as e:
                logging.warning(f"Error during preparation: {str(e)}")
        
        # Handle alias creation/retrieval
        try:
            aliases = bedrock_agent_client.list_agent_aliases(agentId=agent_id)
            alias = next((a for a in aliases['agentAliasSummaries'] if a['agentAliasName'] == 'v1'), None)
            
            if not alias:
                logging.info(f"Creating new alias for agent '{name}'")
                alias = bedrock_agent_client.create_agent_alias(
                    agentId=agent_id,
                    agentAliasName="v1"
                )
                alias_id = alias['agentAlias']['agentAliasId']
            else:
                alias_id = alias['agentAliasId']
                logging.info(f"Using existing alias for agent '{name}'")
            
            logging.info(f"Successfully configured agent '{name}' with ID: {agent_id} and alias: {alias_id}")
            return agent_id, alias_id
            
        except Exception as e:
            logging.error(f"Error managing alias: {str(e)}")
            raise
        
    except Exception as e:
        logging.error(f"Error creating/updating agent: {str(e)}")
        raise RuntimeError(f"Failed to create/update agent: {str(e)}")

def invoke_agent(agent_id, alias_id, input_text, session_id=None):
    """Invoke a Bedrock agent"""
    if session_id is None:
        session_id = str(uuid.uuid4())
        
    response = bedrock_agent_client.invoke_agent(
        agentId=agent_id,
        agentAliasId=alias_id,
        sessionId=session_id,
        inputText=input_text
    )
    
    for event in response['completion']:
        if 'chunk' in event:
            return event['chunk']['bytes'].decode('utf-8')
    
    return None

## Create Embedder Agent

Create the agent responsible for storing documents in the vector store.

In [7]:
embedder_instructions = """
You are an Embedder Agent that handles document storage in the vector store.

Your capabilities include:
1. Adding new documents to the vector store
2. Organizing document metadata
3. Ensuring document quality

Core behaviors:
1. Validate document content before storage
2. Organize metadata appropriately
3. Confirm successful document storage
4. Maintain document quality standards

Response style:
- Be clear about document requirements
- Provide feedback on document quality
- Confirm successful operations
- Guide users on best practices
"""

embedder_functions = [{
    "name": "add_document",
    "description": "Add a new document to the vector store",
    "parameters": {
        "text": {
            "type": "string",
            "description": "The document content to add",
            "required": True
        },
        "metadata": {
            "type": "string",
            "description": "Additional metadata about the document as a JSON string",
            "required": False
        }
    },
    "requireConfirmation": "ENABLED"
}]

embedder_id, embedder_alias = create_agent(
    "embedder",
    embedder_instructions,
    embedder_functions
)

2025-02-19 06:26:55,305 - INFO - Found existing agent 'embedder' with ID: 2IDKSK62PV
2025-02-19 06:26:57,809 - INFO - Agent 2IDKSK62PV reached status: PREPARED
2025-02-19 06:26:58,107 - INFO - Action group already exists for agent 'embedder'
2025-02-19 06:26:58,408 - INFO - Using existing alias for agent 'embedder'
2025-02-19 06:26:58,408 - INFO - Successfully configured agent 'embedder' with ID: 2IDKSK62PV and alias: ZSSXP6ZFDP


## Create Researcher Agent

Create the agent responsible for searching documents.

In [8]:
researcher_instructions = """
You are a Research Assistant that helps users find relevant information in documents.

Your capabilities include:
1. Searching through documents using semantic similarity
2. Providing relevant document excerpts
3. Answering questions based on document content

Core behaviors:
1. Always search available documents before responding
2. Maintain accuracy in document references
3. Provide clear, direct answers with citations
4. Present information in an easy-to-understand manner
5. NEVER invent information not found in documents

Response style:
- Be precise and factual
- Include relevant document excerpts
- Maintain natural conversation flow
- Be concise yet informative
"""

researcher_functions = [{
    "name": "search_documents",
    "description": "Search for relevant documents using semantic similarity",
    "parameters": {
        "query": {
            "type": "string",
            "description": "The search query",
            "required": True
        },
        "k": {
            "type": "integer",
            "description": "Number of results to return",
            "required": False
        }
    },
    "requireConfirmation": "DISABLED"
}]

researcher_id, researcher_alias = create_agent(
    "researcher",
    researcher_instructions,
    researcher_functions
)

2025-02-19 06:26:59,956 - INFO - Found existing agent 'researcher' with ID: YFHVDO09TD
2025-02-19 06:27:00,863 - INFO - Agent YFHVDO09TD reached status: PREPARED
2025-02-19 06:27:01,185 - INFO - Action group already exists for agent 'researcher'
2025-02-19 06:27:01,461 - INFO - Using existing alias for agent 'researcher'
2025-02-19 06:27:01,461 - INFO - Successfully configured agent 'researcher' with ID: YFHVDO09TD and alias: 4NKCOZTDSD


## Create Content Writer Agent

Create the agent responsible for formatting and presenting research findings.

In [9]:
writer_instructions = """
You are a Content Writer Assistant that helps format and present research findings.

Your capabilities include:
1. Formatting research findings in a user-friendly way
2. Creating clear and engaging summaries
3. Organizing information logically
4. Highlighting key insights

Core behaviors:
1. Maintain accuracy of source information
2. Use clear and engaging language
3. Structure content for easy understanding
4. Highlight important points

Response style:
- Use clear, engaging language
- Structure content logically
- Include relevant examples
- Make complex information accessible
"""

writer_functions = [{
    "name": "format_content",
    "description": "Format and present research findings",
    "parameters": {
        "content": {
            "type": "string",
            "description": "The research findings to format",
            "required": True
        },
        "style": {
            "type": "string",
            "description": "The desired presentation style (e.g., summary, detailed, bullet points)",
            "required": False
        }
    },
    "requireConfirmation": "DISABLED"
}]

writer_id, writer_alias = create_agent(
    "writer",
    writer_instructions,
    writer_functions
)

2025-02-19 06:27:03,003 - INFO - Found existing agent 'writer' with ID: 3KPDCSSGGK
2025-02-19 06:27:03,833 - INFO - Agent 3KPDCSSGGK reached status: PREPARED
2025-02-19 06:27:04,256 - INFO - Action group already exists for agent 'writer'
2025-02-19 06:27:04,533 - INFO - Using existing alias for agent 'writer'
2025-02-19 06:27:04,533 - INFO - Successfully configured agent 'writer' with ID: 3KPDCSSGGK and alias: 40PNX6NPGL


## Vector Store Operations

Functions to handle document operations with the vector store.

In [10]:
def add_document(text, metadata=None):
    """Add a document to the vector store"""
    if metadata is None:
        metadata = {}
    elif isinstance(metadata, str):
        try:
            metadata = json.loads(metadata)
        except json.JSONDecodeError:
            metadata = {}
    return vector_store.add_texts([text], [metadata])[0]

def search_documents(query, k=4):
    """Search for similar documents"""
    return vector_store.similarity_search(query, k=k)

## Load Documents

Load documents from external JSON file and add them using the Embedder agent.

In [11]:
# Load documents from JSON file
try:
    with open('documents.json', 'r') as f:
        data = json.load(f)
        documents = data['documents']
        
    # Add each document using the Embedder agent
    for doc in documents:
        embedder_response = invoke_agent(
            embedder_id,
            embedder_alias,
            f'Add this document: {json.dumps(doc)}'
        )
        print(f"Added document: {embedder_response}")
        
    print(f"\nSuccessfully added {len(documents)} documents to vector store")
    
except Exception as e:
    raise ValueError(f"Failed to load documents: {str(e)}")

ValueError: Failed to load documents: 'AgentsforBedrock' object has no attribute 'invoke_agent'

## Example Usage

Demonstrate the multi-agent system in action.

In [ ]:
# Search for information using the researcher agent
researcher_response = invoke_agent(
    researcher_id,
    researcher_alias,
    'What is unique about the Cline AI assistant?'
)
print("Researcher Agent Response:", researcher_response)

# Format the findings using the writer agent
writer_response = invoke_agent(
    writer_id,
    writer_alias,
    f'Format this research finding in a user-friendly way: {researcher_response}'
)
print("\nWriter Agent Response:", writer_response)